# STATS M148 Final Project Code Notebook

### Team Name : Data Crew

**Contributions:**
* Axel Malvaez 
* Darren Hsieh
* Ryan Kawamura
* Taro Iyadomi  
* Dan O’Brien

This is the compilation of all code necessary to obtain the results found in our final report. Necesary files include: 
- Original dataset
- Event definitions dataset
- Smaller sample dataset (optional)

### 0. Utility Functions

In [ ]:
# Import Packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

import warnings
warnings.filterwarnings("ignore")

import random
import torch
import torch.nn as nn
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
def has_correct_sequence(s):
    """Function that checks if the sequence is correct.

    Args:
        s (list of ints): This is the journey steps until end variable in the Fingerhut data.

    Returns:
        Bool : This is a boolean that is True if the sequence is correct and False if it is not.
    """
    
    s = list(s)
    temp = s[0]
    for i in range(1, len(s)):
        if s[i] == temp+1:
            temp = s[i]
        elif s[i] == 1:
            temp = 1
        else :
            print('error because temp is ', temp, ' and x[i] is ', s[i])
            print('i is ', i)
            return False
    return True

def correct_sequences(s):
    """Function that corrects the sequences (journey steps until end) in the Fingerhut data.

    Args:
        s (list of ints): This is the journey steps until end variable in the Fingerhut data.

    Returns:
        seq : This is the corrected journey steps until end variable.
    """
    seq = list(s)
    temp = s[0]
    for i in range(1, len(seq)):
        # if 1 then start again
        if seq[i] == 1:
            temp = 1
        elif seq[i] == temp+1:
            temp = seq[i]
        else :
            seq[i] = temp+1
            temp = seq[i]
    return seq

def fingerhut_data_cleaner(og_df, defs):
    """
    Function to drop duplicates, reindex journey steps, convert timestamps, and merge event definitions.

    args:
     - og_df: This is the original Fingerhut data
     - defs: This is the Event Definitions data frame (also provided by fingerhut)
    
    output:
     - df: This is the cleaned Fingerhut data
    """
    # Dropping duplicate (ignoring journey steps variable)
    df = og_df[['customer_id',
             'account_id',
             'ed_id',
             'event_name',
             'event_timestamp',
             'journey_steps_until_end',
             'journey_id',
             'milestone_number',]]
    
    # Filling in missing milestone numbers with 0
    df.loc[:,['milestone_number']] = df['milestone_number'].copy().fillna(0)

    df = df.drop_duplicates(subset=['customer_id', 'account_id', 'ed_id', 'event_name', 'event_timestamp'])
    df = df.reset_index(drop=True) # re-indexing

    # Re-adding journey_steps_until_end (Axel's way)
    j_steps = df['journey_steps_until_end']
    s_corrected = correct_sequences(j_steps)
    df['journey_steps_until_end'] = s_corrected

    # Convert event_timestamps to datetime objects
    df['event_timestamp'] = pd.to_datetime(df['event_timestamp'], format='mixed')
    
    # Adding a `stage` variable based on the event definitions
    df_stages = defs[['event_name', 'stage']]
    
    df = pd.merge(df, df_stages, on ='event_name', how = 'left')
    
    # Setting positive values for account_ids
    df['account_id'] = remove_if(df, 'account_id')

    # Setting positive values for customer_ids
    df['customer_id'] = remove_if(df, 'customer_id')
    
    return df

def add_n_accounts(df):
    """
    Adds a new column representing the number of accounts each customer has.    
    """
    # Counting the unique number of account_ids for each customer_id
    unique_account_counts = df.groupby('customer_id')['account_id'].nunique().reset_index(name='n_accounts')

    # Merging the unique account counts back into the original dataframe
    return pd.merge(df, unique_account_counts, on='customer_id')

def add_has_discover(df):
    """
    Adds a new column representing whether a customer has gone through the 'Discover' phase.
    """
    discover_customers = df.groupby('customer_id')['stage'].apply(lambda x: 'Discover' in x.values).reset_index(name='has_discover')

    return pd.merge(df, discover_customers, on='customer_id')

def add_has_first_purchase(df):
    """
    Adds a new column representing whether a customer has made their first purchase. 
    
    WE ARE ADDING THE BOOLEAN VALUE WITHOUT TAKING CARE IF IT WAS A MILESTONE OR NOT I.E
    WE ARE NOT TAKING INTO ACCOUNT THAT 'FIRST PURCHASE' COULD BE JUST BROWSING PRODUCTS AND NOT
    ACTUALLY BUYING SOMETHING

    """
    first_purchase_customers = df.groupby('customer_id')['stage'].apply(lambda x: 'First Purchase' in x.values).reset_index(name='has_first_purchase')

    return pd.merge(df, first_purchase_customers, on='customer_id')

def split_sequences(df):
    """Function that given the dataframe, returns a list of lists with the sequences of events

    Args:
        df (dataframe): The dataframe with the data

    Returns:
        result: A list of lists with the sequences of events
    """
    result = []
    current_sequence = []
    
    for idx, step in enumerate(df['journey_steps_until_end']):
        if step == 1:
            # If the list is not empty, i.e. we have a new 1 in
            # the journey we append the current sequence to the result
            if current_sequence:
                result.append(current_sequence)
            current_sequence = [df['ed_id'].iloc[idx]]
        else:
            current_sequence.append(df['ed_id'].iloc[idx])
    
    # In case the last sequence is not empty we append the remaining sequence
    if current_sequence:
        result.append(current_sequence)
    
    return result

def remove_if(df, col_name):
    """Function that removes the negative number of a customer_id

    Args:
        a (str): number of a customer_id

    Returns:
        str : number of a customer_id without the negative sign
    """
    values = df[col_name].apply(lambda x : (-1)*x if x < 0 else x).astype('int64')
    return values

def number_journeys_and_max(cus_df):
    """Function to check the number of journeys in a sequence

    Args:
        seq (list): List of values

    Returns:
        int: Number of journeys in the sequence
    """
    j_steps = cus_df['journey_steps_until_end']
    ones = [i for i, x in enumerate(j_steps) if x == 1]
    return len(ones), max(j_steps)

def has_discover(cust_df):
    """Function to check if a sequence has the discovery event

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        bool: True if the sequence has the discovery event, False otherwise
    """
    return 'Discover' in list(cust_df['stage'])

def number_accounts(cust_df):
    """Function to add the number of accounts to the dataset

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        pd.DataFrame: Dataset with the number of accounts in a new column
    """
    return cust_df['account_id'].nunique()

def has_more_one_journey(j_steps):
    """Function to check if a sequence has repeated values

    Args:
        seq (list): List of values

    Returns:
        bool: True if there are repeated values, False otherwise
    """
    return len(j_steps) != len(set(j_steps))

def most_repeated_event(cust_df):
    """Function that returns the most repeated event in a sequence

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        str: The most repeated event in the sequence
    """
    return cust_df['ed_id'].mode()[0]

def average_length_seq(cust_df):
    """Function to add the average length of the sequences to the dataset

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        pd.DataFrame: Dataset with the average length of the sequences in a new column
    """
    new_df = cust_df.copy()
    # Split the sequences
    sequences = split_sequences(new_df)
    return np.mean([len(seq) for seq in sequences])

def has_prospecting(cust_df):
    """Function to check if a sequence has the prospecting event

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        bool: True if the sequence has the prospecting event, False otherwise
    """
    evnts = list(cust_df['ed_id'])
    return 20 in evnts or 21 in evnts or 24 in evnts

def has_pre_application(cust_df):
    """Function to check if a sequence has the pre-application event

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        bool: True if the sequence has the pre-application event, False otherwise
    """
    return 22 in list(cust_df['ed_id'])

def initial_device(cust_df):
    """Function to get the initial device of a customer
    """
    events = set(cust_df['event_name'])
    phone = ['phone' in event for event in events]
    web = ['web' in event for event in events]
    
    if np.array(phone).any() and np.array(web).any():
        return 3
    elif np.array(phone).any():
        return 1
    elif np.array(web).any():
        return 2
    
def has_approved(cust_df):
    """Function to check if a sequence has the approved event

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        bool: True if the sequence has the approved event, False otherwise
    """
    x = set(cust_df['ed_id'])
    return 15 in x or 12 in x

def get_first_n_events(cust_df, n = 10):
    """Function that returns the first 10 events of a sequence

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        list: The first 10 events of the sequence, padded with np.nan if necessary
    """
    events = cust_df['ed_id'].head(n).tolist()
    # Pad with np.nan if the sequence has fewer than 10 events
    events += [0] * (n - len(events))
    return np.array(events)

def get_time_since_last_event(cust_df, n=10):
    cust_df = cust_df.head(n)
    x = cust_df.groupby(['customer_id', 'journey_id'])['event_timestamp'].diff()
    x = x.fillna(pd.Timedelta(seconds=0))
    x = x.dt.total_seconds()
    x = x.tolist() + [0] * (n - len(x))
    return np.array(x)

def which_milestones(cust_df):
    """Function that returns in a tuple in the following sequence the next statemens:
    - If the customer has applied for credit and it has been approved (milestone 1)
    - If the customer has first purchase (milestone 2)
    - If the customer has account activitation (milestone 3)
    - If the customer has downpayment received (milestone 4)
    - If the customer has downpayment cleared (milestone 5)
    - If the customer has order shipped (milestone 6)

    Args:
        cust_df (_type_): _description_
    """
    milestones = set(cust_df['milestone_number'].unique())
    max_milestone = max(milestones)
    return (1 in milestones, 2 in milestones, 3 in milestones, 4 in milestones, 5 in milestones, 6 in milestones), max_milestone

# Functions for time
def get_idxs(cust_df, stage, milestone = -1):
    """Function to get the indexes of a certain stage

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        list: List with the indexes of a certain stage
    """
    if milestone != -1:
        return list(cust_df[cust_df['milestone_number'] == milestone].index)
    
    return list(cust_df[cust_df['stage'] == stage].index)

def time_in_discover(cust_df, seconds_differences):
    """Function to calculate the time between events

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        list: List with the time between events
    """
    idxs = get_idxs(cust_df, 'Discover')
    
    time_in = []
    for idx in idxs:
        if idx + 1 < len(seconds_differences):
            time_in.append(seconds_differences[idx + 1])
        else:
            time_in.append(0)
    return sum(time_in)

def time_in_apply(cust_df, seconds_differences):
    """Function to calculate the time between events

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        list: List with the time between events
    """
    idxs = get_idxs(cust_df, 'Apply for Credit')
    
    time_in = []
    for idx in idxs:
        if idx + 1 < len(seconds_differences):
            time_in.append(seconds_differences[idx + 1])
        else:
            time_in.append(0)
    return sum(time_in)

def time_reach_milestone1(cust_df, seconds_differences):
    """Function to calculate the time between events

    Args:
        cust_df (pd.DataFrame): Dataset of a certain customer (not all the dataset, just one customer)

    Returns:
        list: List with the time between events
    """
    idxs = get_idxs(cust_df, 'Apply for Credit', 1)
    
    # sum all the times before the milestone
    return sum(seconds_differences[1:idxs[0]+1])

def group_by_approach(cust_df):
    cust_df = cust_df.reset_index(drop=True)
    # applying all the functions to get the data
    num_journeys, max_journey = number_journeys_and_max(cust_df)
    discover = has_discover(cust_df)
    numb_accs = number_accounts(cust_df)
    more_one_journey = has_more_one_journey(cust_df['journey_steps_until_end'])
    repeated_event = most_repeated_event(cust_df)
    avg_length_journey = average_length_seq(cust_df)
    has_pros = has_prospecting(cust_df)
    pre_applic = has_pre_application(cust_df)
    device = initial_device(cust_df)
    x = cust_df['event_timestamp'].diff().dt.total_seconds().tolist()
    time_disc = time_in_discover(cust_df, x)
    time_apply = time_in_apply(cust_df, x)
    # time_milestone1 = time_reach_milestone1(cust_df, x)
    
    milestones, max_milestone = which_milestones(cust_df)
    
    # Creating the new data frame
    new_df = pd.DataFrame({'num_journeys': num_journeys,
                           'max_journey': max_journey,
                           'discover': discover, 
                           'number_accounts': numb_accs,
                           'one_more_journey': more_one_journey,
                           'most_repeated_event': repeated_event,
                           'average_length_seq': avg_length_journey,
                           'approved_credit': milestones[0],
                           'first_purchase': milestones[1],
                           'account_activitation': milestones[2],
                           'downpayment_received': milestones[3],
                           'downpayment_cleared': milestones[4],
                           'order_ships': milestones[5],
                           'max_milestone': max_milestone,
                            'has_prospecting': has_pros,
                            'has_pre_application': pre_applic,
                            'initial_device': device,
                            'time_in_discover': time_disc,
                            'time_in_apply': time_apply,
                            #'time_reach_milestone_1': time_milestone1,
                           'index':[0]})
    return new_df    

def get_classification_dataset(data, event_defs, n_events = 10):
    """This function is the one that gives you the final dataset with the features and the target variable

    Args:
        data (_type_): the original dataset (without any cleaning or anything like that)
        event_defs (_type_): the event definitions dataset
        n_events (int, optional): Number of events in the last column. Defaults to 10.

    Returns:
        df : The final dataset with the features and the target variable
    """
    
    
    df = fingerhut_data_cleaner(data, event_defs)
    # drop the promotion_created event
    idxs = list(df[df['event_name'] == 'promotion_created'].index)
    df.drop(idxs, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Grouping by the customer id and gathering the data
    new_df = df.groupby('customer_id').apply(group_by_approach)
    new_df.drop(columns=['index'], inplace=True)
    
    # Adding the first n events
    x = list(df.groupby('customer_id').apply(get_first_n_events, n = n_events))
    new_df['first_' + str(n_events) + '_events'] = x
    
    # Adding the time of this first n events
    x = list(df.groupby('customer_id').apply(get_time_since_last_event, n = n_events))
    new_df['time_since_last_event'] = x
    
    return new_df

def scaler(col):
    """Function to scale a column"""
    return (col - np.min(col)) / (np.max(col) - np.min(col))

# Generate Embeddings Function
def embeddings(df, col_name = 'first_20_events'):
    sequences_evs = df[col_name].apply(lambda x: np.array(x)).to_numpy()
    sequences_times = df['time_since_last_event'].apply(lambda x: np.array(x)).to_numpy()

    padded_time_events = np.vstack(sequences_evs)
    padded_time_waits = np.vstack(sequences_times)
    
    max_seq_length = 20
    embedding_dim = 5

    model = Sequential()
    model.add(LSTM(units=128, input_shape=(max_seq_length, 1), return_sequences=False))  
    model.add(Dense(units=64, activation='relu'))  
    model.add(Dense(units=32, activation='relu'))  
    model.add(Dense(units=embedding_dim, activation='relu'))  
    model.compile(loss='mse', optimizer='adam')

    print('Predicting embeddings for time...')
    time_embeddings = model.predict(padded_time_waits)

    print('Predicting embeddings for events...')
    event_embeddings = model.predict(padded_time_events)

    event_embd = pd.DataFrame(event_embeddings, columns=[f'event_embd_{i}' for i in range(5)])
    time_embd = pd.DataFrame(time_embeddings, columns=[f'time_embd_{i}' for i in range(5)])

    df.drop(columns=[col_name, 'time_since_last_event'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    new_dfx = pd.concat([df, time_embd, event_embd], axis=1)
    return new_dfx

def vec_to_list(event_list):
    event_list = event_list.replace('[', '').replace(']', '').split()
    event_list = [int(float(x)) for x in event_list]
    return event_list

def preprocessing_steps_embedding(data):
    df = data.copy()
    
    # Dropping columns that introduce bias to the model
    df = df.drop(columns=['Unnamed: 1', 'downpayment_cleared', 'first_purchase',
                          'max_milestone', 'downpayment_received', 'account_activitation', 'customer_id'])
    
    # We set this parameters for future interactions with these features
    number_events_fixed = 20
    col_name = 'first_' + str(number_events_fixed) +'_events'
    
    # As we are reading the data from a csv, the list of events is read as a string
    # and therefore we need to transform this type of data
    result = []
    for item in list(df[col_name]):
        numbers = [int(num) for num in item.replace('[', '').replace(']', '').split()]
        #numbers += [0] * (number_events_fixed - len(numbers))    
        result.append(numbers)
    result2 = []
    for item in list(df['time_since_last_event']):
        numbers = [float(num) for num in item.replace('[', '').replace(']', '').split()]
        #numbers += [0] * (number_events_fixed - len(numbers))    
        result2.append(numbers)
    
    # We have the columns again in a list type
    df[col_name] = result
    df['time_since_last_event'] = result2
    
    # Here we set all the float columns to numbers 0 or 1
    df = df.astype({col: 'float' for col in df.columns[:-2]})
    
    # We realized the dataset in the initial_devices had nan values
    df = df.dropna(axis=0)
    
    # Adding more features
    df['total_time_spent'] = df['time_since_last_event'].apply(lambda x: np.sum(x))
    df['time_mean'] = df['time_since_last_event'].apply(lambda x: np.mean(x))
    print('mean added')
    df['time_std'] = df['time_since_last_event'].apply(lambda x: np.std(x))
    print('std added')
    df['time_max'] = df['time_since_last_event'].apply(lambda x: np.max(x))
    print('max added')
    
    # We create and generate the embeddings
    # we drop the first_20_events and the time_since_last_event column
    # but we kept the embeddings
    df = embeddings(df)

    # Getting the dataset balanced
    df_0, df_1 = df[df.order_ships == 0], df[df.order_ships == 1]
    df_0 = df_0.sample(n=len(df_1), random_state=2024)
    # df_1 = df_1.sample(n=(len(df_0)), replace=True)
    df_balanced = pd.concat([df_0, df_1], axis=0).reset_index(drop=True)

    # shuffle
    df_balanced = df_balanced.sample(frac=1)

    df_X = df_balanced.drop(columns='order_ships')
    target = df_balanced.order_ships
    ori_df = df.drop(columns='order_ships')
    ori_target = df.order_ships

    boolean_col = ['discover', 'one_more_journey', 'approved_credit', 'has_prospecting', 'has_pre_application']

    for col in boolean_col:
        df_X[col] = [1 if val == True else 0 for val in df_X[col]]
        ori_df[col] = [1 if val == True else 0 for val in ori_df[col]]

    return ori_df, ori_target, df_X, target

### 1. Data Exploration

##### a) Small Data Modification

In [1]:
# Load the data
# Change filepaths to appropriate file locations
data_filepath = './export.csv' # smalle_sample.csv for smaller version
event_defs_filepath = './Event+Definitions.csv'

data = pd.read_csv(data_filepath)   # This is the original data
event_defs = pd.read_csv(event_defs_filepath)  # This is the event dictionary

In [ ]:
# Slight Feature Engineering
df = fingerhut_data_cleaner(data, event_defs)
idxs = list(df[df['event_name'] == 'promotion_created'].index)
df.drop(idxs, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby('customer_id').apply(group_by_approach)
df.drop(columns=['index'], inplace=True)

##### b) Order-Shipped Pie Chart

In [ ]:
# Calculate proportions
proportions = df['order_ships'].value_counts(normalize=True) * 100

# Plotting using a pie chart
plt.figure(figsize=(8, 8))  
patches, _, _ = plt.pie(proportions, labels=None, autopct='%1.1f%%', startangle=140)

# Adding a legend
plt.legend(patches, labels=proportions.index, loc="best", fontsize = 14)

# Adding a title
plt.title('Proportion of Order Ships', fontsize = 18)

# Save the plot as an image file
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.savefig('order_ships_pie_chart.png')  # Save as PNG file
plt.show()

##### c) Max Journey Boxplot

In [ ]:
# Plotting using Seaborn with log scale
plt.figure(figsize=(12, 9))  # Adjust the width and height as needed
sns.histplot(df['max_journey'], bins=20, log_scale=True)  # Adjust bins as needed

# Adding labels and title
plt.xlabel('Max Journey Steps (Log Scale)', fontsize = 14)
plt.ylabel('Frequency', fontsize = 14)
plt.title('Histogram of Max Journey Steps (Log Scale)', fontsize = 18)

# Show the plot
plt.savefig('max_journey_histogram.png')
plt.show()

##### d) Discover vs. Order Shipped Barplot

In [ ]:
plot_df = df.copy()
plot_df['discover'] = plot_df['discover'].astype(int)
plot_df['order_ships'] = plot_df['order_ships'].astype(int)

# Calculate proportions manually
proportions = plot_df.groupby('discover')['order_ships'].value_counts(normalize=True).unstack().fillna(0)

# Plotting using Seaborn
plt.figure(figsize=(8, 6))  # Adjust the width and height as needed
bottom_bar = sns.barplot(data=proportions, x=proportions.index, y=proportions[1], color='blue', label='Order Ships')
sns.barplot(data=proportions, x=proportions.index, y=proportions[0], color='orange', bottom=proportions[1], label='No Order Ships')

# Adding labels and title
plt.xlabel('Discover', fontsize = 14)
plt.ylabel('Proportion', fontsize = 14)
plt.title('Relationship between Discover and Order Ships', fontsize = 18)

# Show the legend with custom labels
plt.legend(title='Order Status')

# Set y-axis tick labels
plt.xticks(ticks=[0, 1], labels=['False', 'True'])

# Show the plot

plt.savefig('relationship_discover_order_ships.png')
plt.show()

##### e) YOY Analysis Chart

In [ ]:
order_shipped_df = df[df["event_name"] == "order_shipped"]
yoy_df = order_shipped_df.groupby(['year', 'week']).size().reset_index(name = 'order_shipped')
yoy_df = yoy_df[0:len(yoy_df)-1] 

plt.figure(figsize=(18, 8))

years = yoy_df['year'].unique()
years.sort()

for year in years:
    year_df = yoy_df[yoy_df['year'] == year]
    plt.plot(year_df['week'], year_df['order_shipped'], marker='o', label=str(year))

plt.title('YOY Analysis of Orders Shipped by Week')
plt.xlabel('Week')
plt.ylabel('Number of Orders Shipped')
plt.legend(title='Year', loc='upper left')
plt.grid()
plt.xticks(np.arange(1, 54))
plt.show()

### 2. Data Preparation

##### a) Set Seed

In [ ]:
np.random.seed(2024)

##### b) Feature Engineering

In [ ]:
# Add feature engineered variables (Appendix Table 1)
# Change number_events_fixed to adjust dataset
number_events_fixed = 20
col_name = 'first_' + str(number_events_fixed) +'_events'

df = get_classification_dataset(data, event_defs, n_events=number_events_fixed)
df.reindex(sorted(df.columns), axis=1)

In [ ]:
# Reset index by customer_id
cust_ids = df.index
cust_ids = [x[0] for x in cust_ids]
df.reset_index(drop=True, inplace=True)
df['customer_id'] = cust_ids
df.head()

##### c) Generating Embeddings and Balance Dataset

In [ ]:
# Run embedding and balancing
ori_data, ori_target, df, target = preprocessing_steps_embedding(df)

In [4]:
# Save data (clear memory if memory issues occur)
dataset_filepath = './final_dataset.csv' # Change to your liking
pd.concat([ori_data, ori_target], axis=1).to_csv(dataset_filepath, index=False) # This data was made in order to train the models and test with the whole dataset

### 3. Modeling

##### a) Set Seed

In [ ]:
random.seed(2024)
np.random.seed(2024)
torch.manual_seed(2024)

##### b) Read In Data

In [ ]:
filepath = './final_dataset.csv' # From Data Preparation part
new_dfx = pd.read_csv(filepath)
new_dfx = new_dfx.dropna(axis=0)
num_cols = ['num_journeys', 'max_journey', 'number_accounts', 'average_length_seq', 
            'time_in_discover', 'time_in_apply', 'time_max', 'time_mean', 'time_std', 
            'total_time_spent', 'event_embd_0', 'event_embd_1', 'event_embd_2', 
            'event_embd_3','event_embd_4', 'time_embd_0', 'time_embd_1', 'time_embd_2',
            'time_embd_3', 'time_embd_4']

categorical_cols = ['most_repeated_event', 'initial_device']
boolean_cols = ['discover', 'one_more_journey', 'approved_credit', 'has_prospecting', 'has_pre_application']
target = 'order_ships'
X_train, X_test, y_train, y_test = train_test_split(new_dfx.drop(columns='order_ships'), 
                                                    new_dfx.order_ships, 
                                                    test_size=0.2,
                                                    stratify=new_dfx.order_ships,
                                                    random_state=2024)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

numerical = X_train.loc[:,num_cols]

scaler = StandardScaler()
scaler.fit(numerical)
numerical_tran = scaler.transform(numerical)
numerical_test = scaler.transform(X_test[num_cols])
numerical_tr = pd.DataFrame(numerical_tran, columns=num_cols)
numerical_ts = pd.DataFrame(numerical_test, columns=num_cols)

X_train = pd.concat([numerical_tr, X_train[categorical_cols], X_train[boolean_cols]] , axis=1)
X_test = pd.concat([numerical_ts, X_test[categorical_cols], X_test[boolean_cols]] , axis=1)

y = new_dfx['order_ships'].reset_index(drop=True)
X = new_dfx.drop(columns=['order_ships']).reset_index(drop=True)

##### c) Preliminary Models

In [ ]:
# Cross Validation Function
def cross_val(clf, X, y):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2024)

    f1_scores = []
    accuracy_scores = []

    for train_index, test_index in cv.split(X, y):

        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)

        numerical = X_train.loc[:,num_cols]
        scaler = StandardScaler()
        scaler.fit(numerical)

        numerical_tran = scaler.transform(numerical)
        numerical_test = scaler.transform(X_test[num_cols])
        numerical_tr = pd.DataFrame(numerical_tran, columns=num_cols)
        numerical_ts = pd.DataFrame(numerical_test, columns=num_cols)

        X_train = pd.concat([numerical_tr, X_train[categorical_cols], X_train[boolean_cols]] , axis=1)
        X_test = pd.concat([numerical_ts, X_test[categorical_cols], X_test[boolean_cols]] , axis=1)

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        f1_scores.append(f1_score(y_test, y_pred))
        accuracy_scores.append(accuracy_score(y_test, y_pred))

    print('F1 Score:', np.mean(f1_scores))
    print('F1 Score:', np.std(f1_scores))
    print('Accuracy:', np.mean(accuracy_scores))
    print('Accuracy:', np.std(accuracy_scores)) 

    return np.mean(f1_scores), np.std(f1_scores), np.mean(accuracy_scores), np.std(accuracy_scores)

In [ ]:
# Logistic Regression
log_clf = LogisticRegression(random_state=2024)
log_clf.fit(X_train, y_train)
prediction = log_clf.predict(X_test)
accuracy_score(y_test, prediction)

In [ ]:
# Decision Trees
dt = DecisionTreeClassifier(max_depth=5, random_state=2024)
dt.fit(X_train, y_train)
dt_prediction = dt.predict(X_test)
print(accuracy_score(y_test, dt_prediction))
print(f1_score(y_test, dt_prediction))

In [ ]:
# Ada Boost
ada = AdaBoostClassifier(n_estimators=200, learning_rate=.1, random_state=2024)
ada.fit(X_train, y_train)
ada_prediction = ada.predict(X_test)
print(accuracy_score(y_test, ada_prediction))
print(f1_score(y_test, ada_prediction))

In [ ]:
# Gradient boosting
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=2024)
gb.fit(X_train, y_train)
gb_prediction = gb.predict(X_test)
print(accuracy_score(y_test, gb_prediction))
print(f1_score(y_test, gb_prediction))

In [ ]:
# LGBM
lgbm = LGBMClassifier(n_estimators=200,
                      max_depth=10, 
                      learning_rate=0.1,
                      objective='binary',
                      verbose=-1,
                      random_state=2024)

lgbm.fit(X_train, y_train)
lgbm_prediction = lgbm.predict(X_test)
print(accuracy_score(y_test, lgbm_prediction))
print(f1_score(y_test, lgbm_prediction))

In [ ]:
# XGBoost
xgb_clf = XGBClassifier(n_estimators=200,
                        max_depth=10,
                        learning_rate=0.1,
                        tree_method='hist',
                        objective='binary:logistic',
                        random_state=2024)

xgb_clf.fit(X_train, y_train)
xgb_pred = xgb_clf.predict(X_test)
print(accuracy_score(y_test, xgb_pred))
print(f1_score(y_test, xgb_pred))

In [ ]:
# XGBoost Confusion matrix
cm = confusion_matrix(y_test, xgb_pred)
matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb_clf.classes_)
matrix.plot()
plt.title('Confusion Matrix of XGBoost Before Tuning')
plt.show()

##### d) XGBoost Hyperparameter Tuning

In [ ]:
import optuna

def objective(trial):
    params = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'random_state': 2024,
        'n_estimators': trial.suggest_int('n_estimators', 100, 400),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 5.0)
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train)

    prediction = model.predict(X_test)
    f1 = f1_score(y_test, prediction)

    return f1


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
study.best_params

In [ ]:
best_params = {'n_estimators': 354,
               'max_depth': 10,
               'learning_rate': 0.09777273210428525,
               'subsample': 0.7355818769405801,
               'gamma': 3.513270943451698,
               'scale_pos_weight': 1.7901629274544835,
               'reg_alpha': 0.8063538782719757,
               'reg_lambda': 3.8289679315225817}

In [ ]:
# Train new XGBoost model
xgb_clf = XGBClassifier(**best_params,
                        tree_method='hist',
                        objective='binary:logistic',
                        random_state=2024)

xgb_clf.fit(X_train, y_train)
xgb_pred = xgb_clf.predict(X_test)
print(accuracy_score(y_test, xgb_pred))
print(f1_score(y_test, xgb_pred))

In [ ]:
# Confusion matrix for tuned XGBoost model
cm = confusion_matrix(y_test, xgb_pred)
matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb_clf.classes_)
matrix.plot()
plt.title('Confusion Matrix of Tune XGBoost')
plt.show()

In [ ]:
# Feature Importances
plt.figure(figsize=(10, 6))
plt.barh(xgb_clf.feature_names_in_, xgb_clf.feature_importances_)
plt.xlabel('Feature Importance')
plt.title('Tuned XGBoost Feature Importances')
plt.show()

##### e) Neural Network

In [ ]:
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_train, 
                                                              y_train, 
                                                              test_size=0.2,
                                                              stratify=y_train,
                                                              random_state=2024)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("Class weights:", class_weights_dict)

In [ ]:
model_1 = Sequential([
    Dense(512, activation='relu', input_shape=(27,)),
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.3),  # Dropout layer to prevent overfitting
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.5),  # Dropout layer to prevent overfitting
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(31, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)),
    #Dropout(0.),  # Dropout layer to prevent overfitting
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

learning_rate = 0.001 
optimizer = Adam(learning_rate=learning_rate)

# Compile the model_1
model_1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print the model_1 summary
model_1.summary()

In [ ]:
# Fit the model
history = model_1.fit(X_train_nn, y_train_nn, 
                    epochs=5, 
                    batch_size=128,
                    validation_data=(X_val_nn, y_val_nn),
                    class_weight=class_weights_dict)

In [ ]:
# Make predictions
predictions = model_1.predict(X_test)

# Convert predictions to binary labels
binary_predictions = (predictions > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, binary_predictions.reshape(-1))
print('Accuracy:', accuracy)

f1 = f1_score(y_test, binary_predictions.reshape(-1))
print('F1 Score:', f1)

In [ ]:
plt.figure(figsize=(7, 6))
sns.heatmap(confusion_matrix(y_test, binary_predictions.reshape(-1)), annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Neural Network Confusion Matrix')
plt.show()

### 4. Conclusion

This concludes the analysis and predictive modeling code used for our STATS M148 2024 project. Additional plots and tables were generated using the results above in external software like Tableau and LaTeX. 